In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

# Diccionario que mapea el nombre del animal con su número
mapa_animales = {
    "Delfin": 0, "Ballena": 76, "Carnero": 1, "Toro": 2, "Ciempies": 3, "Alacran": 4,
    "Leon": 5, "Ranas": 6, "Perico": 7, "Raton": 8, "Aguila": 9, "Tigre": 10, "Gato": 11,
    "Caballo": 12, "Mono": 13, "Paloma": 14, "Zorro": 15, "Oso": 16, "Pavo": 17, "Burro": 18,
    "Chivo": 19, "Cochino": 20, "Gallo": 21, "Camello": 22, "Cebra": 23, "Iguana": 24, 
    "Gallina": 25, "Vaca": 26, "Perro": 27, "Zamuro": 28, "Elefante": 29, "Caiman": 30,
    "Lapa": 31, "Ardilla": 32, "Pescado": 33, "Venado": 34, "Jirafa": 35, "Culebra": 36,
    "Tortuga": 37, "Bufalo": 38, "Lechuza": 39, "Avispa": 40, "Canguro": 41, "Tucan": 42,
    "Mariposa": 43, "Chiguire": 44, "Garza": 45, "Puma": 46, "Pavo Real": 47,
    "Puercoespin": 48, "Pereza": 49, "Canario": 50, "Pelicano": 51, "Pulpo": 52, 
    "Caracol": 53, "Grillo": 54, "Oso Hormiguero": 55, "Tiburon": 56, "Pato": 57, 
    "Hormiga": 58, "Pantera": 59, "Camaleon": 60, "Panda": 61, "Cachicamo": 62, 
    "Cangrejo": 63, "Gavilan": 64, "Arana": 65, "Lobo": 66, "Avestruz": 67, "Jaguar": 68, 
    "Conejo": 69, "Bisonte": 70, "Guacamaya": 71, "Gorila": 72, "Hipopotamo": 73, 
    "Turpial": 74, "Guacharo": 75
}

# Leer el archivo de resultados (se asume que el archivo está en el formato adecuado)
with open("resultados_guacharoactivo_2024.txt", "r", encoding="utf-8") as archivo:
    lineas = archivo.readlines()

# Procesar las líneas para extraer la información
datos = []
for linea in lineas:
    # Filtrar las líneas vacías o irrelevantes
    if linea.strip():
        partes = linea.strip().split(",")
        if len(partes) == 3:
            fecha, hora, animalito = partes
            # Homologar el número del animal según el diccionario
            numero = mapa_animales.get(animalito.strip(), None)
            if numero is not None:
                datos.append([fecha, hora, animalito.strip(), numero])

# Crear el DataFrame con los resultados
df_resultados = pd.DataFrame(datos, columns=["Fecha", "Hora", "Animalito", "Número"])

# Mostrar las primeras filas del DataFrame para verificar
print(df_resultados.head())

# Guardar el DataFrame a un archivo CSV
df_resultados.to_csv("resultados_guacharoactivo_completo.csv", index=False, encoding="utf-8")

print("Datos guardados exitosamente en resultados_guacharoactivo_completo.csv")

        Fecha       Hora Animalito  Número
0  2024-01-02   08:00 AM   Ballena      76
1  2024-01-03   08:00 AM   Pescado      33
2  2024-01-04   08:00 AM   Gallina      25
3  2024-01-05   08:00 AM      Lapa      31
4  2024-01-06   08:00 AM     Panda      61
Datos guardados exitosamente en resultados_guacharoactivo_completo.csv


In [3]:
df_resultados.count()

Fecha        21413
Hora         21413
Animalito    21413
Número       21413
dtype: int64

# RandomForestClassifier

In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar los datos
df = pd.read_csv("resultados_guacharoactivo_completo.csv")

# Preprocesamiento
df['Hora'] = df['Hora'].str.strip()
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')
df['Hora'] = pd.to_datetime(df['Hora'], format='%I:%M %p').dt.hour
df['DiaSemana'] = df['Fecha'].dt.dayofweek  # Lunes=0, Domingo=6
df['DiaMes'] = df['Fecha'].dt.day

# Entrenamiento
X = df[['Fecha', 'Hora', 'DiaSemana', 'DiaMes']]
y = df['Número']

X['Fecha'] = (X['Fecha'] - X['Fecha'].min()).dt.days
X = X.dropna()
y = y[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = RandomForestClassifier(
    n_estimators=150,
    criterion='gini',
    max_depth=10,
    min_samples_leaf=3,
    random_state=42,
    class_weight='balanced',
    )
clf.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=3, n_estimators=150, random_state=42)

In [16]:
fecha_elegida = pd.to_datetime("2025-04-10") #Cambiar a la fecha deseada

horas_sorteos = [ 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

dias_desde_inicio = (fecha_elegida - df['Fecha'].min()).days

dia_semana = fecha_elegida.dayofweek
dia_mes = fecha_elegida.day

X_pred = pd.DataFrame({
    'Fecha': [dias_desde_inicio] * len(horas_sorteos),
    'Hora': horas_sorteos,
    'DiaSemana': [dia_semana] * len(horas_sorteos),
    'DiaMes': [dia_mes] * len(horas_sorteos)
})

predicciones = clf.predict(X_pred)

resultados_predichos = pd.DataFrame({
    'Fecha': [fecha_elegida.date()] * len(horas_sorteos),
    'Hora': horas_sorteos,
    'Número_Predicho': predicciones
})

mapa_inverso = {v: k for k, v in mapa_animales.items()}

resultados_predichos['Animalito_Predicho'] = resultados_predichos['Número_Predicho'].map(mapa_inverso)

resultados_predichos

,Fecha,Hora,Número_Predicho,Animalito_Predicho
0,2025-04-10,8,61,Panda
1,2025-04-10,9,61,Panda
2,2025-04-10,10,3,Ciempies
3,2025-04-10,11,61,Panda
4,2025-04-10,12,23,Cebra
5,2025-04-10,13,50,Canario
6,2025-04-10,14,67,Avestruz
7,2025-04-10,15,67,Avestruz
8,2025-04-10,16,38,Bufalo
9,2025-04-10,17,3,Ciempies


In [6]:
y_pred = clf.predict(X_test)


# 1. Accuracy y Error
accuracy = accuracy_score(y_test, y_pred)
error_rate = 1 - accuracy
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"❌ Probabilidad de error: {error_rate:.4f}")

✅ Accuracy: 0.9900
❌ Probabilidad de error: 0.0100
